In [16]:
import json, sqlite3, re, os, datetime
jsonFile = "items.json"
dbFile = "workshop.db"

with open(jsonFile, encoding='utf-8') as fp:
    data = json.load(fp)

In [17]:
con = sqlite3.connect(dbFile)
cur = con.cursor()

con.execute("DROP TABLE IF EXISTS creators;")
con.execute("CREATE TABLE creators (creatorUID INTEGER PRIMARY KEY AUTOINCREMENT, creatorID TEXT, creatorName TEXT, creatorURL TEXT);")

con.execute("DROP TABLE IF EXISTS dlc;")
con.execute("CREATE TABLE dlc (dlcUID INTEGER PRIMARY KEY AUTOINCREMENT, dlcName TEXT);")

con.execute("DROP TABLE IF EXISTS items;")
con.execute("CREATE TABLE items (itemUID INTEGER PRIMARY KEY AUTOINCREMENT, itemID INT, itemName TEXT, itemURL TEXT, rating INT, numVotes INT, creatorUID INT, tags TEXT, size REAL, postedOn date, updatedOn date, description TEXT);")

con.execute("DROP TABLE IF EXISTS dlc_dependency;")
con.execute("CREATE TABLE dlc_dependency (dlcUID INTEGER PRIMARY KEY AUTOINCREMENT, itemID INT, dlcID INT);")

con.execute("DROP TABLE IF EXISTS mod_dependency;")
con.execute("CREATE TABLE mod_dependency (modDependencyUID INTEGER PRIMARY KEY AUTOINCREMENT, itemID INT, dependsOnItemID INT);")

con.commit()

In [18]:
creatorData = {}
for item in data:
    creatorID = item['CreatorID']
    if creatorID not in creatorData:
        creatorName = item['CreatorName']
        creatorURL = item['CreatorURL']
        creatorData[creatorID] = (creatorName, creatorURL)
creatorData = [(key, *value) for key, value in creatorData.items()]

con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.executemany("INSERT INTO creators (creatorID, creatorName, creatorURL) VALUES(?, ?, ?)", creatorData)
con.commit()

In [26]:
dlcData = set()
for item in data:
    for dlc in item['RequiredDLC']:
        dlcData.add(dlc)
dlcData = [(x,) for x in dlcData]

con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.executemany("INSERT INTO dlc (dlcName) VALUES(?)", list(dlcData))
con.commit()

In [20]:
creatorValues = [item['CreatorID'] for item in data]
con = sqlite3.connect(dbFile)
cur = con.cursor()
query = f"SELECT creatorUID, creatorID FROM creators WHERE creatorID IN ({','.join(['?' for _ in creatorValues])})"
cur.execute(query, creatorValues)
creatorUIDs = { key : val for val, key in cur.fetchall() }
con.close()

itemData = []

for item in data:
    try:
        postedOn = datetime.datetime.strptime(item['PostedOn'], "%d %b, %Y @ %I:%M%p").date().strftime("%Y-%m-%d")
    except ValueError:
        try:
            postedOn = datetime.datetime.strptime(item['PostedOn'] + " 2023", "%d %b @ %I:%M%p %Y").date().strftime("%Y-%m-%d")
        except ValueError:
            print(f"Error In PostedOn : {item['PostedOn']}")

    if item["UpdatedOn"] == "Never" :
            updatedOn = postedOn
    else:
        try:
            updatedOn = datetime.datetime.strptime(item['UpdatedOn'], "%d %b, %Y @ %I:%M%p").date().strftime("%Y-%m-%d")
        except ValueError:
            try:
                updatedOn = datetime.datetime.strptime(item['UpdatedOn'] + " 2023", "%d %b @ %I:%M%p %Y").date().strftime("%Y-%m-%d")
            except ValueError:
                print(f"Error In UpdatedOn : {item['UpdatedOn']}")

    match = re.match("([,\.\d]*) MB", item['Size'])
    if match:
        size = float(match.group(1).replace(",", ""))
    else:
        print(f"Error : {item['Name']}")
    
    itemData.append(
        (
            int(item['ID']),
            item['Name'],
            item['URL'],
            int(item['Rating']),
            int(item['NumRating']),
            creatorUIDs[item['CreatorID']],
            item['Tags'],
            size,
            postedOn,
            updatedOn,
            item['Description']
        )
    )

con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.executemany("INSERT INTO items (itemID, itemName, itemURL, rating, numVotes, creatorUID, tags, size, postedOn, updatedOn, description) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", itemData)
con.commit()

In [21]:
modIDs = []
for item in data:
    if len(item['DependsOn']) > 0:
        modIDs.append(item['ID'])
        modIDs.extend([x['ID'] for x in item['DependsOn']])
modIDs = list(set(modIDs))

con = sqlite3.connect(dbFile)
cur = con.cursor()
query = f"SELECT itemUID, itemID FROM items WHERE itemID IN ({','.join(['?' for _ in modIDs])})"
cur.execute(query, modIDs)
itemUIDs = { key : val for val, key in cur.fetchall() }
con.close()


mod_dependency = []

for item in data:
    if len(item['DependsOn']) > 0:
        for x in item['DependsOn'] :
            mod_dependency.append((
                itemUIDs[int(item['ID'])],
                itemUIDs[int(x['ID'])]
            ))

con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.executemany("INSERT INTO mod_dependency (itemID, dependsOnItemID) VALUES(?, ?)", list(mod_dependency))
con.commit()


In [42]:
con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.execute("SELECT * FROM dlc")
dlcUIDs = { key : val for val, key in cur.fetchall() }
con.close()

dlc_dependency = []

itemIDs = [item['ID'] for item in data if len(item['RequiredDLC']) > 0]
con = sqlite3.connect(dbFile)
cur = con.cursor()
query = f"SELECT itemUID, itemID FROM items WHERE itemID IN ({ ','.join(['?' for _ in itemIDs]) })"
cur.execute(query, itemIDs)
itemUIDs = { key : val for val, key in cur.fetchall() }
con.close()


for item in data:
    if len(item['RequiredDLC']) > 0:
        for x in item['RequiredDLC']:
            dlc_dependency.append((
                itemUIDs[int(item['ID'])],
                int( dlcUIDs[x] )
            ))

con = sqlite3.connect(dbFile)
cur = con.cursor()
cur.executemany("INSERT INTO dlc_dependency (itemID, dlcID) VALUES(?, ?)", list(dlc_dependency))
con.commit()
